1-Make connection and cursor

In [6]:
import psycopg2 as p
conn=p.connect(
    host='localhost',
    database='student_management',
    user='postgres',
    password='1212'
)
cur=conn.cursor()
conn.autocommit=True

2-Creating the database

In [2]:
cur.execute('create database student_management')

3-Creating tables

In [12]:
# cur.execute('create table students (student_id serial primary key, name varchar(20), email varchar(255), phone varchar(11))')
# cur.execute('create table courses (course_id serial primary key, name varchar(20), credits int);')
#!multi line statements must use ''' or """
cur.execute('''
	create table enrollments (
		enrollment_id serial primary key,
		student_id int,
		course_id int,
		grade varchar(10),
		foreign key (student_id) references students(student_id) on update cascade on delete cascade,
		foreign key (course_id) references courses(course_id) on update cascade on delete cascade
	)
''')



4-inserting data

In [14]:
cur.execute('''INSERT INTO students (name, email, phone) VALUES
('Alice Johnson', 'alice@example.com', '1234567890'),
('Bob Smith', 'bob@example.com', '9876543210'),
('Charlie Brown', 'charlie@example.com', '5555555555');''')

cur.execute('''INSERT INTO courses (name, credits) VALUES
('Mathematics', 3),
('Computer Science', 4),
('History', 2);''')

cur.execute('''INSERT INTO enrollments (student_id, course_id, grade) VALUES
(1, 1, 'A'),
(1, 2, 'B'),
(2, 3, 'A'),
(3, 2, 'C');''')


5-The querys

In [17]:
cur.execute('select * from students')
for i in cur:
    print(i)

(1, 'Alice Johnson', 'alice@example.com', '1234567890')
(2, 'Bob Smith', 'bob@example.com', '9876543210')
(3, 'Charlie Brown', 'charlie@example.com', '5555555555')
(4, 'Alice Johnson', 'alice@example.com', '1234567890')
(5, 'Bob Smith', 'bob@example.com', '9876543210')
(6, 'Charlie Brown', 'charlie@example.com', '5555555555')


In [21]:
cur.execute("select * from enrollments where grade='A';")
for i in cur:
    print(i)

(1, 1, 1, 'A')
(3, 2, 3, 'A')


In [39]:
cur.execute('select name,credits from courses;')
for i in cur:
    print('course name=>',i[0],'\ncredits->',i[1],'\n')  # i[0] is 'name', i[1] is 'credits'

course name=> Mathematics 
credits-> 3 

course name=> Computer Science 
credits-> 4 

course name=> History 
credits-> 2 



In [49]:
# where s.name='Alice Johnson'
cur.execute('''SELECT s.student_id , s.name , c.name
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
;''')
for i in cur:
    print(i)

(1, 'Alice Johnson', 'Mathematics')
(1, 'Alice Johnson', 'Computer Science')
(2, 'Bob Smith', 'History')
(3, 'Charlie Brown', 'Computer Science')


In [53]:
cur.execute('''
SELECT s.name, COUNT(e.course_id) AS total_courses
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name;''')
for i in cur:
    print('student=',i[0],'courses enrolled in=',i[1])

student= Alice Johnson courses enrolled in= 2
student= Bob Smith courses enrolled in= 1
student= Charlie Brown courses enrolled in= 1


In [57]:
cur.execute('''
SELECT s.name
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.course_id IS NULL;''')
print('students not enrolled in any courses')

for i in cur:
    print('-'+i[0])

students not enrolled in any courses
-Bob Smith
-Charlie Brown
-Alice Johnson


In [59]:
cur.execute('''SELECT * 
FROM enrollments
ORDER BY grade DESC;''')
for i in cur:
    print(i)

(4, 3, 2, 'C')
(2, 1, 2, 'B')
(1, 1, 1, 'A')
(3, 2, 3, 'A')


In [64]:
#grades in numeric values
n_grades = {'A': 4, 'B': 3, 'C': 2, 'D': 1, 'F': 0}

cur.execute('SELECT * FROM enrollments;')
#to put all grades into list for calculating avg
grades = []
for i in cur:
    #!we get the numeric values and put it in grades list to calc its avg
    n_grades_values = n_grades.get(i[3])
    grades.append(n_grades_values)

avggrades = sum(grades)
print('average grades:', avggrades / len(grades))

average grades: 3.25
